## SigNet Package test

**General idea** : test the functions defined in the module1.py file and make a first application.

**General plan** : 
1. Import the data using the DropBox dataset

2. Repeat the clustering several times to get several clustering outcomes

3. Compute the weights of each stocks within each cluster 

4. Compute the returns of each portfolio, where one portfolio corresponds to one clustering and is composed of 5 big assets (which are the 5 clusters) **in this notebook, we first fix the number of clusters to be equal to 5**

Here are the main package we use in this notebook

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from bs4 import BeautifulSoup
import requests 
from pypfopt.efficient_frontier import EfficientFrontier

In [2]:
pip -q install git+https://github.com/robertmartin8/PyPortfolioOpt.git

Note: you may need to restart the kernel to use updated packages.


We also import the module1.py file to use the function we defined there.

In [3]:
## we make some manipulations to correctly import module1

import os
import sys

# on a obtenu le chemin absolu vers notre répertoire en utilisant
# le terminal (ls, pwd, cd)
module_1_directory = '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/code'

# Ajouter le chemin du répertoire 'code' au chemin de recherche Python
sys.path.append(module_1_directory)

# Maintenant, vous pouvez importer module1
import module1

### 1. Importation of data

We download various types of data (Open, High, Low, Close, Adj Close, Volume) for **496** assets between two periods (start, end) into a pandas dataframe denoted as df.

We then store the returns of each of these assets in the 'data'.

In [51]:
df = pd.read_csv('/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv')

In [53]:
data = df[['ticker', 'open', 'close']]

In [54]:
list = data['ticker']
data = data.drop(data.columns[0], axis=1)
data.set_index(list)

,open,close
ticker,,
AA,"[82.0, 80.5, 82.0, 85.875, 86.0, 87.125, 82.0,...","[80.9375, 81.3125, 86.0, 84.875, 84.625, 84.37..."
ABM,"[20.5, 20.125, 20.25, 20.1875, 20.1875, 20.25,...","[20.3125, 20.375, 20.125, 20.1875, 20.25, 20.2..."
ABT,"[35.25, 34.4375, 33.5625, 34.0, 34.5, 36.0, 34...","[35.0, 34.0, 33.9375, 35.125, 35.5, 35.25, 34...."
ADI,"[93.5, 89.5, 85.625, 86.875, 84.0, 90.0, 93.5,...","[90.1875, 85.625, 86.875, 84.5, 86.875, 94.437..."
ADM,"[12.0, 11.8125, 11.875, 11.625, 11.875, 12.0, ...","[12.0, 11.875, 11.6875, 11.75, 11.9375, 11.937..."
...,...,...
XLY,"[31.0, 29.6875, 29.34375, 29.04688, 29.5, 30.5...","[30.125, 29.21875, 28.85938, 29.1875, 30.5625,..."
XOM,"[79.5, 77.375, 78.0, 80.625, 85.9375, 84.1875,...","[78.1875, 76.8125, 81.0, 85.0, 84.9375, 83.75,..."
XRX,"[23.375, 24.1875, 23.0, 23.5625, 23.75, 24.75,...","[24.1875, 23.0625, 23.5, 23.75, 24.3125, 23.75..."


In [114]:
x = np.array(data.iloc[0][0].replace('[', '').replace(']', '').split(', '), dtype=float)
y = np.array(data.iloc[0][1].replace('[', '').replace(']', '').split(', '), dtype=float)
z = pd.DataFrame((x - y)/x)

/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_30968/109333977.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = np.array(data.iloc[0][0].replace('[', '').replace(']', '').split(', '), dtype=float)
/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_30968/109333977.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = np.array(data.iloc[0][1].replace('[', '').replace(']', '').split(', '), dtype=float)


In [129]:
z

,0
0,0.000000
1,0.019048
2,0.009524
3,0.009434
4,0.009259
...,...
5526,-0.003158
5527,-0.005252
5528,-0.001052
5529,0.001057


In [133]:
n, p = df.shape

df2 = pd.DataFrame(index=data.index, columns=['return'])
for i in range(n):
    x = np.array(data.iloc[i][0].replace('[', '').replace(']', '').split(', '), dtype=float)## open
    y = np.array(data.iloc[i][1].replace('[', '').replace(']', '').split(', '), dtype=float) ## close
    z = np.array((y - x) / x)
    df2.iloc[i][0] = z

/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_30968/1938803176.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = np.array(data.iloc[i][0].replace('[', '').replace(']', '').split(', '), dtype=float)## open
/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_30968/1938803176.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = np.array(data.iloc[i][1].replace('[', '').replace(']', '').split(', '), dtype=float) ## close
/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_30968/1938803176.py:8: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future vers

In [137]:
df2.iloc[0, 0]

array([-0.01295732,  0.01009317,  0.04878049, ...,  0.00845594,
       -0.00787534,  0.00709939])

In [136]:
df.iloc[i, 0]

nan